<a href="https://colab.research.google.com/github/Humble-LiuAo/ML-2021/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [1]:
!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!unzip data.zip
!ls 

Downloading...
From: https://drive.google.com/uc?id=1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR
To: /content/data.zip
372MB [00:02, 167MB/s]
Archive:  data.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
data.zip  sample_data  timit_11


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [2]:
import numpy as np

print("Loading data")
data_root='./timit_11/'
train = np.load(data_root+'train_11.npy')
train_label = np.load(data_root+'train_label_11.npy')
test = np.load(data_root+'test_11.npy')

print("Size of training data: {}".format(train.shape))
print("Size of train_label data: {}".format(train_label.shape))
print("Size of testing data: {}".format(test.shape))

Loading data
Size of training data: (1229932, 429)
Size of train_label data: (1229932,)
Size of testing data: (451552, 429)


## Create Dataset

In [3]:
import torch
from torch.utils.data import Dataset

class TIMIDataset(Dataset):
    def __init__(self,X,y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self,idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [4]:
VAL_RATIO = 0

percent = int(train.shape[0]*(1-VAL_RATIO))
print(percent)
# train_x,train_y,val_x,val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
train_x,train_y = train[:percent], train_label[:percent]
indices = [i for i in range(len(train)) if i % 10 == 0]
val_x,val_y = train[indices], train_label[indices]
print('Size of train_x set: {}'.format(train_x.shape))
print('Size of train_y set: {}'.format(train_y.shape))
print('Size of val_x set: {}'.format(val_x.shape))
print('Size of val_y set: {}'.format(val_y.shape))

1229932
Size of train_x set: (1229932, 429)
Size of train_y set: (1229932,)
Size of val_x set: (122994, 429)
Size of val_y set: (122994,)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [5]:
BATCH_SIZE = 200

from torch.utils.data import DataLoader

train_set = TIMIDataset(train_x,train_y)
val_set =TIMIDataset(val_x,val_y)
train_loader = DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set,batch_size=BATCH_SIZE,shuffle=False)

In [6]:
print(len(train_loader))    # train_set/batch_size 长度为batch的个数

6150


Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [7]:
import gc       # Garbage Collector interface

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

0

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [8]:
# 每次输入batch的一个x(1*429)，得到一个y(1*39)
import torch
import torch.nn as nn

class Classfier(nn.Module):
    def __init__(self):
        super(Classfier,self).__init__()
        self.net = nn.Sequential(
                nn.BatchNorm1d(429),
                nn.Dropout(p=0.1),
                nn.Linear(429, 2048),    # Fully connected network
                nn.Sigmoid(),
                nn.BatchNorm1d(2048),         # Batch Normalization(0,1): Accelerating Deep Network Training by Reducing Internal Covariate Shift .
                nn.Dropout(p = 0.5),         # During training, randomly zeroes some of the elements of the input tensor with probability p using samples from a Bernoulli distribution 减少过拟合
                #nn.LeakyReLU(),                 # relu activation
                nn.Linear(2048, 1024),
                nn.Sigmoid(),
                nn.BatchNorm1d(1024),         
                nn.Dropout(p = 0.5),         
                #nn.LeakyReLU(),
                nn.Linear(1024,512),
                nn.Sigmoid(),
                nn.BatchNorm1d(512),       
                nn.Dropout(p = 0.5),
                #nn.LeakyReLU(),
                nn.Linear(512,128),
                nn.BatchNorm1d(128),
                nn.Dropout(p = 0.5),      
                nn.Linear(128,39)     
            )
        self.criterion = nn.CrossEntropyLoss()   # This criterion combines LogSoftmax and NLLLoss in one single class.
        # softmax-->output(只需包含类别)-->crossentropyloss
        # 1轴最大值的下标即类别
        # self.layer1 = nn.Linear(429,1024)
        # self.layer2 = nn.Linear(1024,512)
        # self.layer3 = nn.Linear(512,128)
        # self.BatchNorm = nn.BatchNorm1d(32),         # Batch Normalization(0,1): Accelerating Deep Network Training by Reducing Internal Covariate Shift .
        # self.Dropout = nn.Dropout(p = 0.2),         # During training, randomly zeroes some of the elements of the input tensor with probability p using samples from a Bernoulli distribution 减少过拟合
        # self.act_fn = nn.LeakyReLU()                 # relu activation
        # self.out = nn.linear(128,49)
        # self.act_fn =nn.Sigmoid()

    def forward(self,x):
        # x = self.layer1(x)
        # x = self.act_fn(x)

        # x = self.layer2(x)
        # x = self.act_fn(x)

        # x = self.layer3(x)
        # x = self.act_fn(x)

        # x = self.out(x)

        # return x
        return self.net(x)
    def cal_loss(self, outputs, labels):
        '''Calculate loss'''
        regularization_loss = 0 # 正则项损失
        #for param in model.parameters():
            # may implement L1/L2 regularization here
            # 使用L2正则项
            # regularization_loss += torch.sum(abs(param))
            #regularization_loss += torch.sum(param ** 2)
        return self.criterion(outputs, labels) + 0.00075 * regularization_loss

## Training

In [9]:
# check  device
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [10]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    ##确保输入固定时，输出不变
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [11]:
# fix random seed for reproducibility
same_seeds(0)

# get device
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 100           # number of training epoch
learning_rate = 0.0001      # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classfier().to(device)
# criterion = nn.CrossEntropyLoss()   # This criterion combines LogSoftmax and NLLLoss in one single class.
# softmax-->output(只需包含类别)-->crossentropyloss
# 1轴最大值的下标即类别
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate) # default hyperpapras

DEVICE: cuda


In [12]:
import torch.nn as nn
loss = nn.CrossEntropyLoss()
input = torch.randn(1,2, requires_grad=True)
print(input)
target = torch.empty(1, dtype=torch.long).random_(2)
print(target)
output = loss(input, target)
print(output)
output.backward()

tensor([[-1.8211, -0.7661]], requires_grad=True)
tensor([0])
tensor(1.3537, grad_fn=<NllLossBackward>)


In [13]:
# start training
# 1.Under one epoch, each batch was trained once and recorded batch_loss and pred acc_sum. 
# 2.After the end of one epoch, loss and pred acc_sum were obtained from val_set 
# 3.the cycle(epoches) continued until the end

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    #training
    model.train()
    # yield n(from start which defaults to 0), elem
    for i, data in enumerate(train_loader):
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        batch_loss = model.cal_loss(outputs,labels)
        # torch.max() return max, index
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward()
        optimizer.step()

        # item(): Returns the value of this tensor as a standard Python number.
        # classfication: labels 0-38
        # output 39 dimension(占比最大的下标为类别)， 求出与相等labels的个数(sum): 一个batch的pre准确个数
        # labels Decide in advance which category
        train_acc += (train_pred.cpu()==labels.cpu()).sum().item()
        train_loss += batch_loss.item()
    #训练完一个epoch validation一次
    if len(val_set)>0:
        model.eval()
        with torch.no_grad():
            for i,data in enumerate(val_loader):
                inputs, labels = data
                # 计算的时候在device上，记录/保存转到cpu（）
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = model.cal_loss(outputs, labels) 
                _, val_pred = torch.max(outputs, 1)

                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()
            # train_loss/len(train_loader): sum of batch_loss/sum of batch
            print("[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} Val Acc: {:3.6f} Loss: {:3.6f}".format(
                epoch+1,num_epoch,train_acc/len(train_set),train_loss/len(train_loader),val_acc/len(val_set),val_loss/len(val_loader)
                ) )
            
            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))
# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')

[001/100] Train Acc: 0.461342 Loss: 1.866443 Val Acc: 0.571890 Loss: 1.420421
saving model with acc 0.572
[002/100] Train Acc: 0.530540 Loss: 1.582002 Val Acc: 0.613843 Loss: 1.259169
saving model with acc 0.614
[003/100] Train Acc: 0.557137 Loss: 1.480450 Val Acc: 0.637747 Loss: 1.172284
saving model with acc 0.638
[004/100] Train Acc: 0.574459 Loss: 1.416077 Val Acc: 0.652715 Loss: 1.112471
saving model with acc 0.653
[005/100] Train Acc: 0.587715 Loss: 1.366060 Val Acc: 0.666553 Loss: 1.065708
saving model with acc 0.667
[006/100] Train Acc: 0.598911 Loss: 1.327408 Val Acc: 0.675561 Loss: 1.030471
saving model with acc 0.676
[007/100] Train Acc: 0.607966 Loss: 1.293344 Val Acc: 0.682936 Loss: 1.000476
saving model with acc 0.683
[008/100] Train Acc: 0.616614 Loss: 1.262947 Val Acc: 0.691595 Loss: 0.971247
saving model with acc 0.692
[009/100] Train Acc: 0.623072 Loss: 1.237645 Val Acc: 0.700278 Loss: 0.946420
saving model with acc 0.700
[010/100] Train Acc: 0.629871 Loss: 1.215933 V

In [18]:
a = torch.randn(4,5)
print(a)
a, b =torch.max(a,1)
c = torch.randn(4,5)
_, i = torch.max(c, 1)
# print(a)
# print(b)
print((b==i).sum().item())

tensor([[-0.5602, -1.2086, -0.5289,  0.8673, -0.9513],
        [-0.0882,  0.6169,  0.7564,  0.7262, -0.3078],
        [-1.8331, -0.8385, -2.4585,  1.0640, -0.3778],
        [-2.6254,  0.7161, -0.9207, -0.3473, -1.7305]])
1


## Testing

In [19]:
# create testing dataset
test_set = TIMIDataset(test, None)
test_loader = DataLoader(test_set,batch_size=BATCH_SIZE,shuffle=False)

# create model and load weights from checkpoint
model = Classfier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [20]:
predict = []
model.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1)    # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():   # 先转化为numpy tuple
            predict.append(y)

Write prediction to a CSV file.


In [21]:
with open('prediction.csv', 'w') as f:
    f.write('ID,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))